# Парсинг текста

In [1]:
#Загрузка библиотек
import spacy
import pandas as pd
import numpy as np
import re

In [2]:
#Загрузка датасета
df = pd.read_csv('test_data.csv')
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [3]:
#pip install -U spacy

In [4]:
# Загрузка русской NLP-модели
nlp = spacy.load("ru_core_news_sm")

In [5]:
# Парсинг текста с помощью spaCy. Эта команда запускает целый конвейер
df['text_parsed'] = df['text'].apply(nlp)

In [6]:
df.tail()

,dlg_id,line_n,role,text,text_parsed
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,"(По, поводу, виджетов, и, с, ними, уже, обсуди..."
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,"(Все, я, вам, высылаю, счет, и, с, вами, на, с..."
477,5,140,client,Спасибо спасибо,"(Спасибо, спасибо)"
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,"(Да, да, тогда, созвонимся, ага, спасибо, вам,..."
479,5,142,manager,Ну до свидания хорошего вечера,"(Ну, до, свидания, хорошего, вечера)"


In [7]:
#Выделяем строки с ответами менеджера в отдельный датафрейм
df_manager = df[df['role'].isin(['manager'])]
df_manager

,dlg_id,line_n,role,text,text_parsed
1,0,1,manager,Алло здравствуйте,"(Алло, здравствуйте)"
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"(Меня, зовут, ангелина, компания, диджитал, би..."
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,"(Угу, ну, возможно, вы, рассмотрите, и, другие..."
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,"(Угу, а, на, что, вы, обращаете, внимание, при..."
11,0,11,manager,Что для вас приоритет,"(Что, для, вас, приоритет)"
...,...,...,...,...,...
472,5,135,manager,Так дмитрий,"(Так, дмитрий)"
473,5,136,manager,Все записала тогда завтра ждите звонка,"(Все, записала, тогда, завтра, ждите, звонка)"
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,"(По, поводу, виджетов, и, с, ними, уже, обсуди..."
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,"(Все, я, вам, высылаю, счет, и, с, вами, на, с..."


In [8]:
# Запишем шаблоны регулярных выражений
#приветствие
greeting = r'(добрый.*?день)|(день.*?добрый)|(доброе.*?утро)|(добрый.*?вечер)|(здравствуйте)'
#представление
introdusing = r'(меня.*?зовут)'
#имя менеджера
manager_name = r'(зовут.*?ангелина)|(зовут)|(максим)|(анастасия)'
#наименование компании
company_name = r'(диджитал.*?бизнес)'
#прощание
goodbye = r'(всего.*?доброго)|(до.*?свидания)'

In [9]:
#Для каждой строки ищем шаблон приветствия в столбце «text_parsed».
df_manager['greeting'] = df_manager['text_parsed'].apply(lambda x: True if re.search(greeting, str(x).lower()) else False)

C:\Users\Oleg\AppData\Local\Temp/ipykernel_3312/2205352745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manager['greeting'] = df_manager['text_parsed'].apply(lambda x: True if re.search(greeting, str(x).lower()) else False) #другой вариант: obr = df[' Обращение '].str.lower().str.contains(r)


In [10]:
#Для каждой строки ищем шаблон представлени в столбце «text_parsed».
df_manager['introdusing'] = df_manager['text_parsed'].apply(lambda x: True if re.search(introdusing, str(x).lower()) else False) 

C:\Users\Oleg\AppData\Local\Temp/ipykernel_3312/2515863096.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manager['introdusing'] = df_manager['text_parsed'].apply(lambda x: True if re.search(introdusing, str(x).lower()) else False) #другой вариант: obr = df[' Обращение '].str.lower().str.contains(r)


In [11]:
#Для каждой строки ищем шаблон имени менеджера в столбце «text_parsed».
df_manager['manager_name'] = df_manager['text_parsed'].apply(lambda x: True if re.search(manager_name, str(x).lower()) else False)

C:\Users\Oleg\AppData\Local\Temp/ipykernel_3312/2201697930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manager['manager_name'] = df_manager['text_parsed'].apply(lambda x: True if re.search(manager_name, str(x).lower()) else False) #другой вариант: obr = df[' Обращение '].str.lower().str.contains(r)


In [12]:
#Для каждой строки ищем шаблон наименования компании в столбце «text_parsed».
df_manager['company_name'] = df_manager['text_parsed'].apply(lambda x: True if re.search(company_name, str(x).lower()) else False)

C:\Users\Oleg\AppData\Local\Temp/ipykernel_3312/576957816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manager['company_name'] = df_manager['text_parsed'].apply(lambda x: True if re.search(company_name, str(x).lower()) else False) #другой вариант: obr = df[' Обращение '].str.lower().str.


In [13]:
#Для каждой строки ищем шаблон прощания в столбце «text_parsed».
df_manager['goodbye']= df_manager['text_parsed'].apply(lambda x: True if re.search(goodbye, str(x).lower()) else False)

C:\Users\Oleg\AppData\Local\Temp/ipykernel_3312/695620009.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manager['goodbye']= df_manager['text_parsed'].apply(lambda x: True if re.search(goodbye, str(x).lower()) else False) #другой вариант: obr = df[' Обращение '].str.lower().str.


In [14]:
#Функция проверки условия - менеджер поздоровался и попрощался в диалоге
def greetbye(df):
    greeting_i = False
    goodbye_i = False
    greeting_goodbye = False
    n=0
    for i, row in df.iterrows():
        if row['greeting'] == True:
            greeting_i = True
    for i, row in df.iterrows():    
        if row['goodbye'] == True:
            goodbye_i = True
    if greeting_i == True & goodbye_i == True:
        greeting_goodbye = True
    return greeting_goodbye

In [15]:
#Проверка условия - менеджер поздоровался и попрощался в диалоге 0
dialog_0 = greetbye(df_manager[df_manager.dlg_id == 0])
dialog_0

True

In [16]:
#Проверка условия - менеджер поздоровался и попрощался в диалоге 1
dialog_1 = greetbye(df_manager[df_manager.dlg_id == 1])
dialog_1

True

In [17]:
#Проверка условия - менеджер поздоровался и попрощался в диалоге 2
dialog_2 = greetbye(df_manager[df_manager.dlg_id == 2])
dialog_2 

False

In [18]:
#Проверка условия - менеджер поздоровался и попрощался в диалоге 3
dialog_3 = greetbye(df_manager[df_manager.dlg_id == 3])
dialog_3

True

In [19]:
#Проверка условия - менеджер поздоровался и попрощался в диалоге 4
dialog_4 = greetbye(df_manager[df_manager.dlg_id == 4])
dialog_4

False

In [20]:
#Проверка условия - менеджер поздоровался и попрощался в диалоге 5
dialog_5 = greetbye(df_manager[df_manager.dlg_id == 5])
dialog_5

False

In [23]:
#Заполняем столбец 'greeting_goodbying' p+значениями True если менеджер поздоровался и попрощался в диалоге, False если нет.
for i in range(6):
    df_manager.loc[df_manager.dlg_id == i, 'greeting_goodbying'] = greetbye(df_manager[df_manager.dlg_id == i])

C:\Users\Oleg\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [26]:
#Итоговый датасет
df_manager

,dlg_id,line_n,role,text,text_parsed,greeting,introdusing,manager_name,company_name,goodbye,greeting_goodbying
1,0,1,manager,Алло здравствуйте,"(Алло, здравствуйте)",True,False,False,False,False,True
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"(Меня, зовут, ангелина, компания, диджитал, би...",False,True,True,True,False,True
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,"(Угу, ну, возможно, вы, рассмотрите, и, другие...",False,False,False,False,False,True
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,"(Угу, а, на, что, вы, обращаете, внимание, при...",False,False,False,False,False,True
11,0,11,manager,Что для вас приоритет,"(Что, для, вас, приоритет)",False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
472,5,135,manager,Так дмитрий,"(Так, дмитрий)",False,False,False,False,False,False
473,5,136,manager,Все записала тогда завтра ждите звонка,"(Все, записала, тогда, завтра, ждите, звонка)",False,False,False,False,False,False
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,"(По, поводу, виджетов, и, с, ними, уже, обсуди...",False,False,False,False,False,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,"(Все, я, вам, высылаю, счет, и, с, вами, на, с...",False,False,False,False,False,False


In [28]:
#Сохраняем результат парсинга в CSV
df_manager.to_csv('test_data_parsed.csv')